In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import sklearn.decomposition as decompose
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.preprocessing import MinMaxScaler
Images_path = "C:\\Users\\Ahmad Beltagy\\PycharmProjects\\Sheet2\\orl_faces"

In [2]:
def readImages():
    matrix = []
    labels = []
    for i in range(1, 41, 1):
        for j in range(1, 11, 1):
            img = mpimg.imread(Images_path+"\\s" + str(i) + "\\" + str(
                j) + "" + ".pgm", format="pgm")
#             print("image is ",img)
            row = np.reshape(img, 10304)
#             print("row is ",row)
            labels.append(i)
            matrix.append(row)
    return matrix , labels

In [3]:
def splitIntoTrainingAndTesting(matrix,labels):
    trainingSet = []
    trainingLabels = []
    testingSet = []
    testingLabels = []

    for testing in range(0, 400, 2):
        testingSet.append(matrix[testing])
        testingLabels.append(labels[testing])

    for training in range(1, 401, 2):
        trainingSet.append(matrix[training])
        trainingLabels.append(labels[training])

    return trainingSet , trainingLabels , testingSet , testingLabels



In [4]:
def chooseEigenValues(eigenValues , alpha):
    total = sum(eigenValues)
    counter = 0
    explainedVariance = 0
    while(explainedVariance <= alpha):
        counter = counter + 1
        explainedVariance = sum(eigenValues[0:counter]) / total
        #print(explainedVariance)
    return counter

In [5]:
def PCA(dataMatrix , alpha):
    # mean = np.mean(dataMatrix, axis=0)
    cov = np.cov(dataMatrix.transpose() , bias=True)
    #print("Covariance matrix \n",cov)
    eigenValues , eigenVectors = np.linalg.eigh(cov)
    
    index = eigenValues.argsort()[::-1]
    
    eigenValues = eigenValues[index]
    
    numberOfEigenValues = chooseEigenValues(eigenValues, alpha)
    
    print("number of chosen eigenValues is ",numberOfEigenValues)
    
    desiredVectors = eigenVectors.transpose()
    desiredVectors = desiredVectors[index][:numberOfEigenValues]
    desiredVectors = desiredVectors.transpose()
    
    return desiredVectors

In [6]:
def PCA_Accuaracy(trainingSet, trainingLabels, testingSet, testingLabels, projectionMatrix):
    
    TrainingSetProjection = np.dot(trainingSet , projectionMatrix)
    TestingSetProjection  = np.dot(testingSet , projectionMatrix) 
    print("projection of training Set size is : ",len(TrainingSetProjection)," x ",len(TrainingSetProjection[0]))
    
    neigh = KNeighborsClassifier(n_neighbors=1)
    
    neigh.fit(TrainingSetProjection,trainingLabels)
    
    #print(neigh.score(X=TrainingSetProjection,y=trainingLabels, sample_weight=None))
    test_pred = neigh.predict(TestingSetProjection)
    
    print('confusion matrix ',confusion_matrix(testingLabels, test_pred))  
    print(classification_report(testingLabels, test_pred))
    print()

In [7]:
def LDA(dataMatrix):
    totalMean = np.mean(dataMatrix,axis=0)
    temparray = []
    meanList = []
    CenteredData = []
    meanDiff = []
    SMatrix = []
    S = []
    print()
    #calculates mean for each class
    for i in range(0,40,1):
        temparray.append(dataMatrix[i*5])
        temparray.append(dataMatrix[i*5 +1])
        temparray.append(dataMatrix[i*5 +2])
        temparray.append(dataMatrix[i*5 +3])
        temparray.append(dataMatrix[i*5 +4])
        meanList.append(np.mean(np.array(temparray), axis=0))
        meanDiff.append(meanList[i] - totalMean)
        SMatrix.append(5*np.outer(meanDiff,meanDiff))
        temparray = []
        meanDiff = []
        
        
    B = sum(SMatrix)
   
    
    
    #Centers the Data
    for j in range(0,40,1):
        CenteredData.append(dataMatrix[j*5] - meanList[j])
        CenteredData.append(dataMatrix[j*5 +1] - meanList[j])
        CenteredData.append(dataMatrix[j*5 +2] - meanList[j])
        CenteredData.append(dataMatrix[j*5 +3] - meanList[j])
        CenteredData.append(dataMatrix[j*5 +4] - meanList[j])
        
    CenteredData = np.array(CenteredData)
    
    return B , CenteredData    

In [8]:
def calculateEigenLDA(S, B):
    sInverse = np.linalg.pinv(S)
    
    print()
    eigenValues , eigenVectors = np.linalg.eig(np.matmul(sInverse,B))
    
    
    index = eigenValues.argsort()[::-1]
    
    eigenValues = eigenValues[index]
    print("sorted eigenValues \n",eigenValues)
    
    numberOfEigenValues = 39
    
    print(eigenVectors)
    
    desiredVectors = eigenVectors.transpose()
    print(desiredVectors)
    desiredVectors = desiredVectors[index][:numberOfEigenValues]
    desiredVectors = desiredVectors.transpose()
    print(desiredVectors)
    return desiredVectors

In [9]:
def LDA_Accuaracy(trainingSet, trainingLabels, testingSet, testingLabels, projectionMatrix):
    
    TrainingSetProjection = np.dot(trainingSet , projectionMatrix)
    TestingSetProjection  = np.dot(testingSet , projectionMatrix) 
    print("projection of training Set size is : ",len(TrainingSetProjection)," x ",len(TrainingSetProjection[0]))
    
    neigh = KNeighborsClassifier(n_neighbors=1)
    
    neigh.fit(TrainingSetProjection,trainingLabels)
    
    #print(neigh.score(X=TrainingSetProjection,y=trainingLabels, sample_weight=None))
    test_pred = neigh.predict(TestingSetProjection)
    
    print(confusion_matrix(testingLabels, test_pred))  
    print(classification_report(testingLabels, test_pred))
    print()

In [10]:
#main fn 
dataMatrix, Labels = readImages()

#getting training set and testing Set
trainingSet, trainingLabels, testingSet, testingLabels = splitIntoTrainingAndTesting(dataMatrix,Labels)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(trainingSet) 
scaler  = scaler.fit(trainingSet) 
trainingSet = scaler.transform(trainingSet)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(testingSet) 
scaler  = scaler.fit(testingSet) 
testingSet = scaler.transform(testingSet)

# PCA

In [18]:
alpha = [0.8,0.85,0.9,0.95]

for i in alpha:
    print("for alpha",i)
    projectionMatrix = PCA(np.array(trainingSet), i)
    PCA_Accuaracy(trainingSet,trainingLabels,testingSet,testingLabels,projectionMatrix)

for alpha 0.8
number of chosen eigenValues is  35
projection of training Set size is :  200  x  35
confusion matrix  [[3 1 0 ... 0 0 0]
 [0 5 0 ... 0 0 0]
 [0 0 5 ... 0 0 0]
 ...
 [0 0 0 ... 5 0 0]
 [0 0 0 ... 0 5 0]
 [0 0 0 ... 0 0 2]]
              precision    recall  f1-score   support

           1       1.00      0.60      0.75         5
           2       0.83      1.00      0.91         5
           3       0.83      1.00      0.91         5
           4       1.00      1.00      1.00         5
           5       0.62      1.00      0.77         5
           6       1.00      1.00      1.00         5
           7       0.71      1.00      0.83         5
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         5
          10       1.00      0.80      0.89         5
          11       0.83      1.00      0.91         5
          12       1.00      1.00      1.00         5
          13       1.00      1.00      1.00         5
      

# LDA

In [19]:
B , CenteredData = LDA(np.array(trainingSet))
S = np.zeros((10304,10304))
tempMatrix = []
temp = np.array([])

print("B MATRIX \n",B)

for j in range(0,40,1):
    tempMatrix.append(CenteredData[j*5])
    tempMatrix.append(CenteredData[j*5+1])
    tempMatrix.append(CenteredData[j*5+2])
    tempMatrix.append(CenteredData[j*5+3])
    tempMatrix.append(CenteredData[j*5+4])
    tempMatrix  = np.array(tempMatrix)
    S = S + np.dot(tempMatrix.transpose(),tempMatrix)
    temp = np.array([])
    tempMatrix = []

print("S MATRIX and dimensions ",len(S),"x",len(S[0]),"\n",S)    
projectionMatrix = calculateEigenLDA(S, B)
LDA_Accuaracy(trainingSet,trainingLabels,testingSet,testingLabels,np.real(projectionMatrix))


B MATRIX 
 [[13.3008265  12.40961126 13.20372209 ... -1.53122996 -0.80784039
  -0.58240453]
 [12.40961126 11.60247372 12.32560284 ... -1.3690459  -0.70891897
  -0.51409522]
 [13.20372209 12.32560284 13.14507067 ... -1.41604199 -0.70504932
  -0.49405978]
 ...
 [-1.53122996 -1.3690459  -1.41604199 ...  8.03169455  7.37121988
   7.29313095]
 [-0.80784039 -0.70891897 -0.70504932 ...  7.37121988  7.22705997
   7.08650647]
 [-0.58240453 -0.51409522 -0.49405978 ...  7.29313095  7.08650647
   7.09802519]]
S MATRIX and dimensions  10304 x 10304 
 [[ 1.61098988  1.44660588  1.54742495 ... -0.14105547 -0.13070796
   0.02006253]
 [ 1.44660588  1.43095418  1.46742388 ... -0.0902315  -0.08248016
   0.06255179]
 [ 1.54742495  1.46742388  1.62238679 ... -0.12314513 -0.03436314
   0.127157  ]
 ...
 [-0.14105547 -0.0902315  -0.12314513 ...  3.11574404  1.94810854
   1.49967775]
 [-0.13070796 -0.08248016 -0.03436314 ...  1.94810854  2.44058426
   2.02809528]
 [ 0.02006253  0.06255179  0.127157   ...  1.